# Guidance Transformation class to normalize and schedule Classifier-free Guidance.

In [ ]:
#|default_exp normalizers

The following classes implement:  

1. Baseline Classifier-free Guidance  
2. Scaling the prediction by the ratio of norms  
3. Scaling the `t - u` update by its norms  
4. Scaling both 2. + 3.

In [ ]:
#|export

import torch

In [ ]:
#| export

class GuidanceTfm:
    """Baseline Classifier-free Guidance for Diffusion.
    """
    name = "CFGuidance"
    def __init__(self):
        # the unconditioned and conditioned latents
        self.u, self.t = None, None
        # the guidance update
        self.diff = None
        # the final prediction
        self.pred = None

    def apply_cfg(self, guidance_scale):
        self.pred = self.u + guidance_scale * (self.diff)

    def pre_process(self):  pass
    def post_process(self): pass

    def set_u(self, u): self.u = u
    def set_t(self, t): self.t = t
    def set_latents(self, u, t):
        self.u, self.t = u, t

    def compute_update(self):
        self.diff = self.t - self.u

    def set_pred(self, pred): self.pred = pred
    def get_pred(self): return self.pred
    
    
class PredNormGuidance(GuidanceTfm):
    """Scales the noise prediction by its overall norm.
    """
    name = "BaseNormGuidance"
    def post_process(self):
        self.pred = self.pred * (torch.linalg.norm(self.u) / torch.linalg.norm(self.pred))
        
        
class TNormGuidance(GuidanceTfm):
    """Scales the latent mix of `t - u`.

    Note: Roughly equivalent to the GuidedTTS Norm
        Reference: https://arxiv.org/pdf/2205.15370.pdf
    """
    name = "TNormGuidance"
    def pre_process(self):
        self.diff = (self.diff / torch.linalg.norm(self.diff)) * torch.linalg.norm(self.u)
        
        
class FullNormGuidance(TNormGuidance, PredNormGuidance):
    "Applies both Prediction and T-Norm on the noise prediction."
    name = "FullNormGuidance"


'''Map from string name to guidance normalization class.
'''
name2norm = {
    'no_norm': GuidanceTfm,
    'pred_norm': PredNormGuidance,
    't_norm': TNormGuidance,
    'full_norm': FullNormGuidance,
}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()